## Performing face detection on a still image

In [7]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2

In [9]:
# define face detection function
face_cascade = cv2.CascadeClassifier('C:/Users/acer/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
# read still image and convert to Grayscale
img = cv2.imread('../sci2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            
# Detect face

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                                      
# draw rectangle RDI
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w, y+h),(255,0,0),3)
                                      
# show face detected image
cv2.imshow('Actor Detected!', img)
cv2.waitKey(0)
cv2.destoryAllWindows()
                                      

AttributeError: module 'cv2' has no attribute 'destoryAllWindows'

## Performing real-time face detection on a camera

In [1]:
import cv2

In [2]:
face_cascade = cv2.CascadeClassifier('C:/Users/acer/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0)
fps = 15 #frame per sec
size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))

videoWriter = cv2.VideoWriter('', cv2.VideoWriter_fourcc('I', '4', '2', '0'), fps, size)

print(cv2.data.haarcascades)

while(True):
    ret,frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for(x,y,w,h) in faces:
        img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.imshow("camera", frame)
    videoWriter.write(frame)
    if cv2.waitKey(1) & 0xff == ord("q"):
        break
camera.release()
cv2.destoryAllWindows()

C:\Users\acer\anaconda3\lib\site-packages\cv2\data\


AttributeError: module 'cv2' has no attribute 'destoryAllWindows'

## Image segmentation

1. load the image, and create a mask populated with zeros with the same shape as the image loaded;

In [8]:
import numpy as np
import cv2
img = cv2.imread('jack.webp')
mask = np.zeros(img.shape[:2], np.uint8)

2. create zero-filled foreground and background models

In [9]:
bgdModel = np.zeros((1,65), np.float64)
fgdModel = np.zeros((1,65), np.float64)

3. init the GrabCut algorithm with a rectangule identifying the subject to isolate and run the GrabCut algorithm specifying the empty models and mask

In [10]:
rect = (20, 20, 470, 800)
cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[6.13090543e-01, 2.47111365e-02, 2.69919749e-01, 8.23359492e-02,
         9.94262166e-03, 2.03573922e+02, 2.08064242e+02, 2.09804967e+02,
         1.57022541e+02, 1.64060243e+02, 1.68761620e+02, 1.73365104e+02,
         1.83614296e+02, 1.87528626e+02, 2.28469077e+02, 2.33066874e+02,
         2.34302401e+02, 1.92949926e+01, 2.24323252e+01, 2.40917204e+01,
         2.73727515e+00, 2.23264799e+00, 2.01232276e+00, 2.23264799e+00,
         2.70939834e+00, 2.25105176e+00, 2.01232276e+00, 2.25105176e+00,
         2.12255046e+00, 2.72826341e+03, 2.77965833e+03, 2.81014720e+03,
         2.77965833e+03, 2.84775540e+03, 2.88008199e+03, 2.81014720e+03,
         2.88008199e+03, 2.92362261e+03, 4.66521154e+02, 4.04598499e+02,
         3.69629795e+02, 4.04598499e+02, 3.5

4. After this, the values in the 'mask' between 0 and 3. The values, 0 and 2, will be converted into zeros, and 1-3 into onese

In [11]:
mask2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')

5. multiply the original image with mask2

In [12]:
img_seg = img*mask2[:,:,np.newaxis]
cv2.imshow('Original', img)
cv2.imshow('GrabCut', img_seg)
cv2.waitKey()
cv2.destoryAllWindows()

AttributeError: module 'cv2' has no attribute 'destoryAllWindows'

## Image segmentation with Watershed Algorithm

Load image and convert grayscale

In [2]:
import numpy as np
import cv2
img = cv2.imread('jack.webp')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Thresholding

In [3]:
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

Removing Noise using Morphology Transformation

In [4]:
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=3)

Apply dilation to get sure background

In [5]:
sure_bg = cv2.dilate(opening, kernel, iterations=5)

Apply distance transformation to get sure foreground

In [6]:
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

Find unknown area

In [7]:
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

Build barrier to merge

In [8]:
ret, markers = cv2.connectedComponents(sure_fg)

Unknow stays at zeros and add 1 to background

In [9]:
markers = markers+1
markers[unknown==255] = 0

Fill water and draw barrier in blue color

In [10]:
markers = cv2.watershed(img, markers)
img[markers == -1] = [255, 0, 0]
cv2.imshow('Watershed_image', img)